In [1]:
import datetime
import hashlib
import hmac
import requests
import pandas as pd

#prod
# key = "5v524tpr1cc2t83k3bb8e6lu8uh19cm1"
# secret = "aqfr1drih5k1ttdi6jq7oej9355ru37s"
#daily
key = "v5pb1uoi36tf5b6knmf2ppikihn7nmja"
secret = "u477q85huo8h02vummmi0c37g6mu9r7a"


# def create_headers():

#     method = "GET"
#     path = "/assetmanagement/list/offline"
#     timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
#     string_to_sign: str = method + " " + path + timestamp
#     local_secret: bytes = secret.encode("utf-8")
#     string_to_sign: bytes = string_to_sign.encode("utf-8")
#     signature = hmac.new(local_secret, string_to_sign, hashlib.sha256).hexdigest()
#     auth: str = key + ":" + signature
#     return {
#         "content-type": "application/json",
#         "x-hp-hmac-authentication": auth,
#         "x-hp-hmac-date": timestamp,
#         "x-hp-hmac-algorithm": "SHA256",
#     }

def make_request():
    headers = create_headers()
    print("Headers:", headers)
    proxies = {
        "https": "http://proxy-txn.austin.hpicorp.net:8080/"
    }
    url = "https://daily.gsbprint.net/api/kcService/assetmanagement/list/offline?size=250&date=01/01/2019%2000:00:00&from=245"
    #url = "https://daily.gsbprint.net/knowledge-zone/#/view/asset/46182"
    #url = "https://www.printos.com/api/kcService/assetmanagement/list/offline?size=5&date=01/01/2019%2000:00:00&from=0"
    #https://stg.printopt.org
    response = requests.get(url, headers=headers, proxies=proxies)
    data = response.json()
    return data


# Call the make_request function
data = make_request()

# https://www.printos.com/knowledge-zone/#/view/asset/{documentID}

NameError: name 'create_headers' is not defined

In [2]:
pd.set_option('display.max_columns', None)  # Show all columns
# pd.reset_option('display.max_rows', None)     # Show all rows
# pd.set_option('display.width', 1000)        # Set the display width to a larger number
# pd.set_option('display.max_colwidth', None) # Show full content of each column

In [148]:
# base_url = "https://daily.gsbprint.net/api/kcService/assetmanagement/list/offline"
# headers = create_headers()
# proxies = {
#         "https": "http://proxy-txn.austin.hpicorp.net:8080/"
#     }
# params = {
#         "size": 250,
#         "date": "01/01/2019 00:00:00",
#         "bu": "Indigo",
#         "from": 0
#     }
# all_data = []
# response = requests.get(base_url, headers=headers, proxies=proxies, params=params)
# data = response.json()
# members = data['kzOfflineAssetList']['members']
# print(data)
import requests

base_url = "https://www.printos.com/api/kcService/assetmanagement/list/offline"
headers = create_headers()
proxies = {
    "https": "http://proxy-txn.austin.hpicorp.net:8080/"
}

params = {
    "size": 250,  # Number of records to fetch per request
    "date": "01/01/2019 00:00:00",
    "bu": "Indigo",
    "from": 0  # Start from the first record
}

all_data = []
total = 1  # Initialize with a value greater than 0 to enter the loop

while params['from'] < total:
    try:
        response = requests.get(base_url, headers=headers, proxies=proxies, params=params)

        # Check if the response was successful
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code}. Skipping this batch.")
            params['from'] += params['size']
            continue

        data = response.json()

        # Check if 'kzOfflineAssetList' exists in the response
        if 'kzOfflineAssetList' not in data:
            print(f"'kzOfflineAssetList' not found in response. Skipping this batch.")
            params['from'] += params['size']
            continue

        # Fetch the total number of records only once
        if params['from'] == 0:
            total = data['kzOfflineAssetList']['total']

        # Extract the members and add them to all_data
        members = data['kzOfflineAssetList']['members']
        all_data.extend(members)

        # Increment the 'from' parameter to fetch the next set of records
        params['from'] += params['size']

        print(f"Retrieved {len(all_data)} out of {total} records")

    except Exception as e:
        print(f"An error occurred: {e}. Skipping this batch.")
        params['from'] += params['size']

print(f"Total records retrieved: {len(all_data)}")



Retrieved 250 out of 12079 records
Retrieved 500 out of 12079 records
Retrieved 750 out of 12079 records
Retrieved 1000 out of 12079 records
Retrieved 1250 out of 12079 records
Retrieved 1500 out of 12079 records
Retrieved 1750 out of 12079 records
Retrieved 2000 out of 12079 records
Retrieved 2250 out of 12079 records
Retrieved 2500 out of 12079 records
Retrieved 2750 out of 12079 records
Retrieved 3000 out of 12079 records
Retrieved 3250 out of 12079 records
Retrieved 3500 out of 12079 records
Retrieved 3750 out of 12079 records
Retrieved 4000 out of 12079 records
Retrieved 4250 out of 12079 records
Retrieved 4500 out of 12079 records
Retrieved 4750 out of 12079 records
Retrieved 5000 out of 12079 records
Retrieved 5250 out of 12079 records
Retrieved 5500 out of 12079 records
Retrieved 5750 out of 12079 records
Retrieved 6000 out of 12079 records
Retrieved 6250 out of 12079 records
Retrieved 6500 out of 12079 records
Retrieved 6750 out of 12079 records
Retrieved 7000 out of 12079 rec

In [3]:
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
import datetime
#from datetime import datetime
import hashlib
import hmac
import pandas as pd
import re
import json
import requests
import logging

In [4]:
class OpenSearchConnectConfig:
    def __init__(self):
        self.host = 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com'       
        self.port = 443
        self.auth = ('kcs-admin', '!QAZ2wsx')    
        self.use_ssl = True
        self.verify_certs = False

class OpenSearchConnectConfigDev:
    def __init__(self):
        self.host = 'vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com' 
       # https://vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com/
        self.port = 443
        self.auth = ('ks-stg-admin', 'K3!0dg_7Hc')    
        self.use_ssl = True
        self.verify_certs = False
        
class OpenSearchConnectConfigStg:
    def __init__(self):
        self.host = 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com' 
       # https://vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com/
        self.port = 443
        self.auth = ('ks-stg-admin', 'K3!0dg_7Hc')    
        self.use_ssl = True
        self.verify_certs = False

In [366]:
# key = "v5pb1uoi36tf5b6knmf2ppikihn7nmja"
# secret = "u477q85huo8h02vummmi0c37g6mu9r7a"

# def create_headers():

#     method = "GET"
#     path = "/assetmanagement/list/offline"
#     timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
#     string_to_sign: str = method + " " + path + timestamp
#     local_secret: bytes = secret.encode("utf-8")
#     string_to_sign: bytes = string_to_sign.encode("utf-8")
#     signature = hmac.new(local_secret, string_to_sign, hashlib.sha256).hexdigest()
#     auth: str = key + ":" + signature
#     return {
#         "content-type": "application/json",
#         "x-hp-hmac-authentication": auth,
#         "x-hp-hmac-date": timestamp,
#         "x-hp-hmac-algorithm": "SHA256",
#     }

import requests
import pandas as pd

def make_api_request():
    headers = create_headers()
    proxies = {
    "https": "http://proxy-txn.austin.hpicorp.net:8080/"
    }

    base_url = 'https://www.printos.com/api/kcService/assetmanagement/list/offline'#"https://daily.gsbprint.net/api/kcService/assetmanagement/list/offline"
    params = {
        "size": 250,
        "date": "01/01/2019 00:00:00",
        "bu": "Indigo",
        "from": 0
    }

    all_data = []

    while True:
        response = requests.get(base_url, headers=headers, proxies=proxies, params=params)
        data = response.json()
        members = data['kzOfflineAssetList']['members']
        all_data.extend(members)
        
        # Check if we've retrieved less than the maximum size, indicating we're done
        if len(members) < params['size']:
            break
        
        # Increment the 'from' parameter
        params['from'] += params['size']
    
    df = pd.json_normalize(all_data, sep='_')
    df.columns = [col.replace('assetResponse_', '') for col in df.columns]
    return df
test=make_api_request()

In [367]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12067 entries, 0 to 12066
Data columns (total 46 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   actionEnum                                     12067 non-null  object 
 1   id                                             12067 non-null  int64  
 2   name                                           12067 non-null  object 
 3   ratingResponse_rating                          12067 non-null  float64
 4   ratingResponse_number_of_votes                 12067 non-null  int64  
 5   provider                                       12067 non-null  object 
 6   format                                         12067 non-null  object 
 7   language                                       12067 non-null  object 
 8   uuid                                           12067 non-null  object 
 9   preSignedUrl                                   120

In [5]:
def fetch_opensearch_data(client,documentID,index_name ):
    query = {
        "query": {
            "match": {
                "documentID": documentID
            }
        }
    }
    result = client.search(index=index_name, body=query)
    item_list = [hit for hit in result['hits']['hits']]
    return item_list

In [130]:

def index_data_to_opensearch(df, index_name):
        # Connect to OpenSearch
        config1 = OpenSearchConnectConfigStg()
        timeout_seconds = 160
        client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs,
            timeout=timeout_seconds
        )
        
        dict_list = df.to_dict(orient='records')
        print(dict_list[0])
        # Initialize counters and lists
        successful_records = 0
        failed_records = []
        new_records = 0
        updated_records = 0

        # Prepare actions for bulk indexing with upsert
        actions = []
        for record in dict_list:

            action = {
                "_index": index_name,
                "_id": record["documentID"],
                "pipeline": "doccebo.v2.emb_pipeline",
                "_op_type": "update",
                "doc": record,
                "doc_as_upsert": True, 
            }
            actions.append(action)

        # Perform bulk indexing with upsert
        try:
            success, _ = bulk(client, actions=actions)
            successful_records = success
        except Exception as e:
            # Log and store failed records
            for record in dict_list:
                failed_records.append((record, str(e), 'KZ_API'))
                logging.error(f"Failed to index document {record}: {e}  : 'KZ_API_INGRESS'")
        
        # Store failed records and update fromDate if necessary
        if failed_records:
            logging.info(f"Indexing completed with {len(failed_records)} failed records.")
            #self.store_failed_records(failed_records)
        else:
            # Update fromDate attribute if all records are successful
            current_datetime = datetime.datetime.now()
            #KzAPI.fromDate = current_datetime.strftime('%Y-%m-%d')
            logging.info("All records indexed successfully. Updating fromDate.")
        
        # Log indexing statistics to 'kas_log_index' index
        try:
            data_to_index = {
                "successful_records": successful_records,
                "failed_records": len(failed_records),
                "new_records": new_records,
                "updated_records": updated_records,
                "timestamp": datetime.datetime.now(),
                'Source': 'KZ_API_DATA_INGESTION',
                'index_loaded': index_name
            }
            client.index(index='kz_log_index', body=data_to_index)
            logging.info("Successfully indexed data to kz_log_index.")
        except Exception as e:
            logging.error(f"Failed to index data to kz_log_index: {e}")
        logging.info('successful_records: %s ::: failed_records: %s', successful_records, failed_records)
        return successful_records, len(failed_records)

In [131]:
index_data_to_opensearch(df1,'kz.v6-embed')

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


{'documentID': 11774, 'title': '7600_BLUEBABY_CMYKW', 'provider': 'Storage-Service', 'format': 'pdf', 'language': 'English', 'uuid': 'c65d581e-0e85-4807-a13c-9158b86a0467', 'description': 'White Ink Demo File', 'contentType': 'Document', 'Domain': 'Indigo', 'contentUpdateDate': '2019-03-25 09:42:44', 'namedAsset': 'c65d581e-0e85-4807-a13c-9158b86a0467', 'lowestRolesList': 'Channel', 'catalog_number': '', 'asset_state': {'name': 'Regular', 'numericOrder': 99, 'dueDate': None}, 'publisher': {'id': 1, 'name': 'HP Indigo'}, 'ratingResponse': {'number_of_votes': 0, 'rating': 0.0}, 'products': ['HP Indigo 7600 Digital Press', 'Series 3'], 'persona': 'Engineer', 'renderLink': 'https://www.printos.com/knowledge-zone/#/view/asset/11774', 'ti_desc_prod': '7600_BLUEBABY_CMYKW White Ink Demo File', 'Doc_Status': 'published'}


C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghr

(9721, 0)

In [ ]:
print(create_headers)

In [6]:
def kz_get_pwp_device_lookup():
    df = pd.read_csv('kz_pwp_products.csv')
    df['device_names'] = df['device_names'].apply(lambda x: x.split(','))
    product_to_device = dict(zip(df['product_code'], df['device_names']))
    return product_to_device


In [7]:
def fetch_existing_documents(client, index_name, document_ids):
    try:
        response = client.mget(index=index_name, body={"ids": document_ids})
        return {doc["_id"]: doc["found"] for doc in response["docs"]}
    except Exception as e:
        logging.error(f"Failed to fetch existing documents: {e}")
        return {}


In [8]:
def create_headers():
    key = "5v524tpr1cc2t83k3bb8e6lu8uh19cm1"
    secret = "aqfr1drih5k1ttdi6jq7oej9355ru37s"
    method = "GET"
    path = "/assetmanagement/list/offline"
    timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
    string_to_sign: str = method + " " + path + timestamp
    local_secret: bytes = secret.encode("utf-8")
    string_to_sign: bytes = string_to_sign.encode("utf-8")
    signature = hmac.new(local_secret, string_to_sign, hashlib.sha256).hexdigest()
    auth: str = key + ":" + signature
    return {
        "content-type": "application/json",
        "x-hp-hmac-authentication": auth,
        "x-hp-hmac-date": timestamp,
        "x-hp-hmac-algorithm": "SHA256",
    }

In [9]:
header=create_headers()
print(header)

{'content-type': 'application/json', 'x-hp-hmac-authentication': '5v524tpr1cc2t83k3bb8e6lu8uh19cm1:f1623c65e4b0ccb830fca01f4ed00a3c4a824f18b67752436fa705a04106ea9f', 'x-hp-hmac-date': '2024-10-07T07:07:08Z', 'x-hp-hmac-algorithm': 'SHA256'}


In [ ]:
def make_api_request(self):
        headers = create_headers()
        proxies = {
            "https": "http://proxy-txn.austin.hpicorp.net:8080/"
        }
        base_url = "https://www.printos.com/api/kcService/assetmanagement/list/offline"
        params = {
            "size": 250,
            "date": "01/01/2019 00:00:00",
            "bu": "Indigo",
            "from": 0
        }
        
        all_data = []
        total = 500  # Initialize to enter the loop
        while params['from'] < total:
            try:
                response = requests.get(base_url, headers=headers, proxies=proxies, params=params)
                # Check if the response was successful
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code}. Skipping this batch.")
                    params['from'] += params['size']
                    continue
                data = response.json()

                if params['from'] == 0:
                    total = data['kzOfflineAssetList']['total']
    
                # Extract the members and add them to all_data
                members = data['kzOfflineAssetList']['members']
                all_data.extend(members)
    
                # Increment the 'from' parameter to fetch the next set of records
                params['from'] += params['size']
   
            except Exception as e:
                print(f"An error occurred: {e}. Skipping this batch.")
                params['from'] += params['size']
    
        # Convert the collected data into a pandas DataFrame
        df = pd.json_normalize(all_data, sep='_')
        df.columns = [col.replace('assetResponse_', '') for col in df.columns]
        
        return df

In [10]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

class kzloader:
    failed_records_df = pd.DataFrame(columns=['Record', 'Error', 'Source'])
    #date: "01/01/2019 00:00:00"
    def make_api_request(self):
        headers = create_headers()
        proxies = {
            "https": "http://proxy-txn.austin.hpicorp.net:8080/"
        }
        base_url = "https://www.printos.com/api/kcService/assetmanagement/list/offline"
        params = {
            "size": 250,
            "date": "01/01/2019 00:00:00",
            "bu": "Indigo",
            "from": 0
        }
    
        # Initial request to get the total number of records
        initial_response = requests.get(base_url, headers=headers, proxies=proxies, params=params)
        initial_data = initial_response.json()
        total_records = initial_data['kzOfflineAssetList']['total']
        all_data = initial_data['kzOfflineAssetList']['members']
    
        def fetch_data(start):
            batch_params = params.copy()
            batch_params['from'] = start
            response = requests.get(base_url, headers=headers, proxies=proxies, params=batch_params)
            response.raise_for_status()
            return response.json()['kzOfflineAssetList']['members']
    
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(fetch_data, start) for start in range(params['size'], total_records, params['size'])]
            for future in as_completed(futures):
                all_data.extend(future.result())
    
        df = pd.json_normalize(all_data, sep='_')
        df.columns = [col.replace('assetResponse_', '') for col in df.columns]
        
        return df
        
    def preprocess_data(self, df):
        # Create the 'asset_state' column
        df['asset_state'] = df.apply(lambda row: {
            'name': row['asset_state_name'],
            'numericOrder': row['asset_state_numericOrder'],
            'dueDate': row['asset_state_dueDate']
        }, axis=1)
        language_code_mapping = {
                'English': 'en',
                'EN': 'en',
                'En': 'en',
                'Chinese': 'zh',
                'French': 'fr',
                'German': 'de',
                'Japanese': 'ja',
                'Korean': 'ko',
                'Portuguese': 'pt',
                'Russian': 'ru',
                'Spanish': 'es',
                'Italian': 'it',
                'Simplified_Chinese': 'zh-CN',
                'Portuguese-Br': 'pt-BR',
                'Hebrew': 'he',
                'Spanish_Latam': 'es-419',
                'Hungarian': 'hu',
                'Dutch': 'nl',
                'Other Languages': 'xx'  # Handling other unspecified languages
            }
        
        # Strip whitespace and normalize case
        df['language'] = df['language'].str.strip().str.title()
        
        # Replace the language names with their corresponding ISO codes
        df['language'] = df['language'].replace(language_code_mapping)
        # Create the 'publisher' column
        df['publisher'] = df.apply(lambda row: {
            'id': row['publisher_id'],
            'name': row['publisher_name']
        }, axis=1)
        
        # Create the 'ratingResponse' column
        df['ratingResponse'] = df.apply(lambda row: {
            'number_of_votes': row['ratingResponse_number_of_votes'],
            'rating': row['ratingResponse_rating']
        }, axis=1)
        
        # lambda Function to extract first names from tag group
        product_to_device=kz_get_pwp_device_lookup()
        df['products']=df['tags_groups'].apply(lambda tag_groups: [{'name': tag['name'], 'type': tag['type']} for group in tag_groups for tag in group['tags_names']] if isinstance(tag_groups, list) else [])
        df['products']=df['products'].apply(lambda products: [p['name'] for p in products if p['type'] in ['Press Model', 'Press Series','Platforms ']])  
        df['products'] = df['products'].apply(
            lambda products_list: [item for product in products_list for item in [product] + product_to_device.get(product, [product])]
        )
        df['products'] = df['products'].apply(lambda x: list(set(x)))
        df.loc[(df['bu'] == 'PWP') & (df['tags_groups'].isna()), 'products'] = df.loc[(df['bu'] == 'PWP') & (df['tags_groups'].isna()), 'products'].apply(
        lambda x: x + ['PWP-Generic'] if isinstance(x, list) else ['PWP-Generic']
        )
        # Filter the DataFrame (if needed)
        df = df[df['bu'].isin(['Indigo', 'Pwi','PWP'])].copy()
        df['bu']=df['bu'].replace('Pwi','PWP')
        # Rename columns
        df.rename(columns={
            'id': 'documentID',
            'bu': 'Domain',
            'body': 'description',
            'name': 'title',
            'fileType': 'contentType',
            'modification_date': 'contentUpdateDate',
            'lowestRoles_lowestRole_roleName': 'lowestRolesList'
        }, inplace=True)

        df['persona'] = df['lowestRolesList'].apply(lambda role: 'Operator' if role in ['Press Operator', 'PSP Admin'] 
                                                    else 'Engineer' if role in ['HP CE', 'Channel','KZ Admin'] 
                                                    else 'NOROLES')

        # Drop the original separate columns
        columns_to_drop=[
            'actionEnum', 'created_by', 'notification_date', 'size', 'assetGroup', 
            'upload_date', 'asset_state_name', 'header', 'tags_groups', 'projectID', 
            'keyWordResponses','isIndexable','isVisible', 'isActive', 'notificationProperties_isFilteredByDeviceType', 
            'asset_state_numericOrder', 'ratingResponse_rating', 'asset_state_dueDate', 
            'lowestRoles_lowestRole_id','lowestRoles_lowestRolesSet','lowestRoles_lowestRole_name',
            'lowestRoles_lowestRole_level', 'lowestRoles_lowestRole_bu', 
            'publisher_id', 'publisher_name', 'ratingResponse_number_of_votes', 
            'notificationProperties_isRealTimeNotification', 
            'notificationProperties_isWeeklyReport','roles','thumbnail_url','lowestRolesSet','original_name'
        ]
        df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
        df['contentUpdateDate'] = pd.to_datetime(df['contentUpdateDate']).dt.strftime('%Y-%m-%d %H:%M:%S')
 
        render_url = "https://www.printos.com/knowledge-zone/#/view/asset"
        # Create the 'url' column
        df['renderLink'] = df['documentID'].apply(lambda x: f"{render_url}/{x}")

        #df["Doc_Status"] = df["ti_desc_prod"].apply(lambda x: "rejected" if x == "N/A" else "published")
        
        # Fill NaN values
        df.fillna('', inplace=True)
        df['description']=df['description'].apply(lambda x: re.sub(r'(<img.*?>|<iframe.*?>|<.*?>|\xa0)', '', x).replace('\n', ' ').strip())
        df['description']=df.apply(lambda row: row['description'] + ' ' + str(row['catalog_number']) if str(row['catalog_number']) not in row['description'] else row['description'], axis=1)
        df['ti_desc_prod'] = df['title'].fillna('') + ' ' + df['description'].fillna('')
        df['ti_desc_prod'] = df['ti_desc_prod'].replace(r'^\s*$', 'N/A', regex=True)
        df['ti_desc_prod']=df['ti_desc_prod'].apply(lambda x: re.sub(r'<.*?>', '', x))
        df["Doc_Status"] = df.apply(
            lambda x: "rejected" if x["ti_desc_prod"] == "N/A" or x["persona"] == "NOROLES" else "published", axis=1
        )
        c500_records=df[df['products'].apply(lambda x:any('C5' in i.upper() for i in x ))]['documentID']
        df=df[~df['documentID'].isin(c500_records)]
        return df

    def index_data_to_opensearch(self, df, index_name):
        # Connect to OpenSearch
        config1 = OpenSearchConnectConfigStg()
        timeout_seconds = 160
        client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs,
            timeout=timeout_seconds
        )
    
        dict_list = df.to_dict(orient="records")
        document_ids = [record["documentID"] for record in dict_list]
        
        # Batch fetch existing documents
        existing_docs = fetch_existing_documents(client, index_name, document_ids)
    
        # Initialize counters and lists
        successful_records = 0
        failed_records = []
        new_records = 0
        updated_records = 0
    
        # Prepare actions for bulk indexing with upsert
        actions = []
        for record in dict_list:
            if existing_docs.get(record["documentID"], False):
                updated_records += 1
            else:
                new_records += 1
    
            action = {
                "_index": index_name,
                "_id": record["documentID"],
                "pipeline": "doccebo.v2.emb_pipeline",
                "_op_type": "update",
                "doc": record,
                "doc_as_upsert": True,
            }
            actions.append(action)


        # Perform bulk indexing with upsert
        try:
            success, _ = bulk(client, actions=actions)
            successful_records = success
            print(success)
        except Exception as e:
            # Log and store failed records
            for record in dict_list:
                failed_records.append((record, str(e), 'kzloader_API'))
                logging.error(f"Failed to index document {record}: {e}  : 'kzloader_API'")
        
        # Store failed records and update fromDate if necessary
        if failed_records:
            logging.info(f"Indexing completed with {len(failed_records)} failed records.")
            self.store_failed_records(failed_records)
        else:
            # Update fromDate attribute if all records are successful
            current_datetime = datetime.datetime.now()
            kzloader.fromDate = current_datetime.strftime('%Y-%m-%d')
            logging.info("All records indexed successfully. Updating fromDate.")
        
        # Log indexing statistics to 'doccebo_course_delta_log_index' index
        try:
            data_to_index = {
                "successful_records": successful_records,
                "failed_records": len(failed_records),
                "new_records": new_records,
                "updated_records": updated_records,
                "timestamp": datetime.datetime.now(),
                'Source': 'kzloader_DATA_INGESTION',
                'index_loaded': index_name
            }
            client.index(index='kzloader_log_index', body=data_to_index)
            logging.info("Successfully indexed data to kzloader_log_index.")
        except Exception as e:
            logging.error(f"Failed to index data to kzloader_log_index: {e}")
        logging.info('successful_records: %s ::: failed_records: %s', successful_records, failed_records)
        return successful_records, len(failed_records)

    def store_failed_records(self, failed_records):
        try:
            # Check if the DataFrame is empty
            if len(kzloader.failed_records_df) == 0:
                # Initialize failed_records_df if it's empty
                kzloader.failed_records_df = pd.DataFrame(columns=['Record', 'Error', 'Source'])
                
            # Convert failed_records to DataFrame
            failed_records_df_new = pd.DataFrame(failed_records, columns=['Record', 'Error', 'Source'])
            
            # Concatenate new records with existing DataFrame
            kzloader.failed_records_df = pd.concat([kzloader.failed_records_df, failed_records_df_new], ignore_index=True)
            
            # Log success message
            logging.info(f"Failed records stored successfully. Total failed records: {len(kzloader.failed_records_df)}")
            
            # Return updated DataFrame
            return kzloader.failed_records_df
        except Exception as e:
            # Log error if storing failed records fails
            logging.error(f"Failed to store failed records: {e}")
            return None
                
    def kz_job(self):
        try:
            df = self.make_api_request()
            df = self.preprocess_data(df)
            result = self.index_data_to_opensearch(df, index_name="kz.v6-embed")
            return result
        except Exception as e:
            logging.error("An error occurred during kz_job: %s", str(e))
            raise

In [ ]:
def generate_embeddings(text):    
    # Make sure to `pip install openai` first    
    client = OpenAI(base_url="http://15.43.5.213:11434/v1", api_key="ollama")    
# client = OpenAI(base_url="http://pngindia.auth.hpicorp.net:444/v1", api_key="lm-studio")   
    model = "nomic-embed-text:latest"    text = text.replace("\n", " ")    
    return client.embeddings.create(input=[text], model=model).data[0].embeddingclass CustomEmbeddingFunction(EmbeddingFunction):    def __call__(self, texts: Documents) -> Embeddings:        return list(map(generate_embeddings, texts))    def embed_query(self, query: str) -> List[float]:        return generate_embeddings(query)    def embed_documents(self, texts: Documents) -> Embeddings:        return list(map(generate_embeddings, texts))docsearch = OpenSearchVectorSearch(    embedding_function=CustomEmbeddingFunction(),    opensearch_url="vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com:443",    http_auth = ("ks-stg-admin", "K3!0dg_7Hc"),    timeout=300,    use_ssl=True,    verify_certs=False,    ssl_assert_hostname=False,    ssl_show_warn=False,    index_name="knowledge-ph2-1",)docs = docsearch.similarity_search("String", k=5)

In [11]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

def make_api_request(self):
    headers = create_headers()
    proxies = {
        "https": "http://proxy-txn.austin.hpicorp.net:8080/"
    }
    base_url = "https://www.printos.com/api/kcService/assetmanagement/list/offline"
    params = {
        "size": 250,
        "date": "01/01/2019 00:00:00",
        "bu": "Indigo",
        "from": 0
    }

    # Initial request to get the total number of records
    initial_response = requests.get(base_url, headers=headers, proxies=proxies, params=params)
    initial_data = initial_response.json()
    total_records = initial_data['kzOfflineAssetList']['total']
    all_data = initial_data['kzOfflineAssetList']['members']

    def fetch_data(start):
        batch_params = params.copy()
        batch_params['from'] = start
        response = requests.get(base_url, headers=headers, proxies=proxies, params=batch_params)
        response.raise_for_status()
        return response.json()['kzOfflineAssetList']['members']

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data, start) for start in range(params['size'], total_records, params['size'])]
        for future in as_completed(futures):
            all_data.extend(future.result())

    df = pd.json_normalize(all_data, sep='_')
    df.columns = [col.replace('assetResponse_', '') for col in df.columns]
    
    return df
import time
start=time.time()
check=make_api_request()
end=time.time()
print(end-start)


81.26152300834656


In [12]:
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12106 entries, 0 to 12105
Data columns (total 46 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   actionEnum                                     12106 non-null  object 
 1   id                                             12106 non-null  int64  
 2   name                                           12106 non-null  object 
 3   ratingResponse_rating                          12106 non-null  float64
 4   ratingResponse_number_of_votes                 12106 non-null  int64  
 5   provider                                       12106 non-null  object 
 6   format                                         12106 non-null  object 
 7   language                                       12106 non-null  object 
 8   uuid                                           12106 non-null  object 
 9   preSignedUrl                                   120

In [11]:
result = kzloader()

In [12]:
df=result.make_api_request()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12134 entries, 0 to 12133
Data columns (total 46 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   actionEnum                                     12134 non-null  object 
 1   id                                             12134 non-null  int64  
 2   name                                           12134 non-null  object 
 3   ratingResponse_rating                          12134 non-null  float64
 4   ratingResponse_number_of_votes                 12134 non-null  int64  
 5   provider                                       12134 non-null  object 
 6   format                                         12134 non-null  object 
 7   language                                       12134 non-null  object 
 8   uuid                                           12134 non-null  object 
 9   preSignedUrl                                   120

In [14]:
df1=result.preprocess_data(df)

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9781 entries, 0 to 12133
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   documentID         9781 non-null   int64 
 1   title              9781 non-null   object
 2   provider           9781 non-null   object
 3   format             9781 non-null   object
 4   language           9781 non-null   object
 5   uuid               9781 non-null   object
 6   preSignedUrl       9781 non-null   object
 7   description        9781 non-null   object
 8   contentType        9781 non-null   object
 9   Domain             9781 non-null   object
 10  contentUpdateDate  9781 non-null   object
 11  namedAsset         9781 non-null   object
 12  lowestRolesList    9781 non-null   object
 13  catalog_number     9781 non-null   object
 14  asset_state        9781 non-null   object
 15  publisher          9781 non-null   object
 16  ratingResponse     9781 non-null   object
 17 

In [16]:
result.index_data_to_opensearch(df1,index_name="kz.v6-embed")

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urlli

9781


C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


(9781, 0)

In [282]:
# pwp_kz=pd.read_excel(r"C:\Users\KuBP064\HP\docebo\Device-Series-Pressname mapping data from LearnDB.xlsx",sheet_name='kz')
# pwp_kz[['Device/Category Name','Series Alias Name']]

In [281]:
#pwp_kz.groupby('Series Alias Name')['Device/Category Name'].apply(list).to_dict()


In [110]:
### make csv
# dfp = pd.DataFrame(list(product_to_device.items()), columns=['product_code', 'device_names'])

# # Convert the lists of device names into comma-separated strings
# dfp['device_names'] = dfp['device_names'].apply(lambda x: ','.join(x))

# # Save the DataFrame as a CSV file
# dfp.to_csv('docebo_pwp_products.csv', index=False)


In [47]:
dff=pd.read_csv('docebo_pwp_products.csv')
dff.to_dict(orient='records')

[{'product_code': 'T11XX', 'device_names': 'HP PageWide T1100 Press series'},
 {'product_code': 'T2XX',
  'device_names': 'HP PageWide Web Press T200 Color series,HP PageWide Web Press T200 HD series,HP PageWide Web Press T230 Color series,HP PageWide Web Press T260 Mono'},
 {'product_code': 'A22XX',
  'device_names': 'HP PageWide Advantage 2200 Series'},
 {'product_code': 'T3XX',
  'device_names': 'HP PageWide Web Press T300 HD series,HP PageWide Web Press T300 Mono series,HP PageWide Web Press T300 series,HP PageWide Web Press T350,HP PageWide Web Press T350M,HP PageWide Web Press T360'},
 {'product_code': 'T4XX',
  'device_names': 'HP PageWide T400S Press,HP PageWide Web Press T400 Color series,HP PageWide Web Press T400 HD series,HP PageWide Web Press T400 Mono series,HP PageWide Web Press T410'}]

In [145]:
# result.index_data_to_opensearch(df1,index_name="kz.v6-embed")

In [15]:
class OpenSearchConnectConfig:
    def __init__(self):
        self.host = 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com'       
        self.port = 443
        self.auth = ('kcs-admin', '!QAZ2wsx')    
        self.use_ssl = True
        self.verify_certs = False
config1=OpenSearchConnectConfig()
client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs
        )

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [96]:
#index_name = 'kz.v6-embed'
index_name = 'kaas.v2.dev'
query = {
    'query': {
        'match_all': {}
    }
}

size = 1000  # Number of documents to fetch per request
start = 0
data = []
total_hits = size  # Initialize to size to start the loop


while start < total_hits:
    response = client.search(
        index=index_name,
        body=query,
        size=size,
        from_=start
    )
    
    hits = response['hits']['hits']
    total_hits = response['hits']['total']['value'] if start == 0 else total_hits
    data.extend(hit['_source'] for hit in hits)
    start += size

# Step 4: Create a DataFrame from the list of documents
df = pd.DataFrame(data)
# Create a DataFrame from the list of documents
kaas_demo = pd.DataFrame(data)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com

In [17]:
import pandas as pd

class DataQualityProcessor:

    @staticmethod
    def get_value(row, column):
        """ Helper method to safely get a value from a row, returning an empty string if the column is missing. """
        return row[column] if column in row else ''

    @staticmethod
    def generate_remarks(row, include_id=False, include_catalog_number=False):
        remarks = {
            'documentID': DataQualityProcessor.get_value(row, 'documentID'),
            'title': DataQualityProcessor.get_value(row, 'title'),
            'products': DataQualityProcessor.get_value(row, 'products'),
            'persona': DataQualityProcessor.get_value(row, 'persona'),
            'description': DataQualityProcessor.get_value(row, 'description'),
            'contentUpdateDate': DataQualityProcessor.get_value(row, 'contentUpdateDate'),
            'Domain': DataQualityProcessor.get_value(row, 'Domain'),
            'Doc_Status': DataQualityProcessor.get_value(row, 'Doc_Status'),
            'language': DataQualityProcessor.get_value(row, 'language'),
            'Title_is_too_short': 'N',
            'Description_is_too_short': 'N',
            'Domain_is_empty': 'N',
            'Title_is_empty': 'N',
            'Description_is_empty': 'N',
            'Disclosure_Level_is_empty': 'N',
            'Persona_is_empty': 'N',
            'Products_is_empty': 'N',
            'catalog_number_is_empty': 'N'
        }
        
        # Assign 'id' based on include_id flag
        remarks['id'] = DataQualityProcessor.get_value(row, 'id') if include_id else None
    
        # Assign 'catalog_number' based on include_catalog_number flag
        remarks['catalog_number'] = DataQualityProcessor.get_value(row, 'catalog_number') if include_catalog_number else None
    
        # Check if catalog_number is empty
        if 'catalog_number' in row and row['catalog_number'] == '':
            remarks['catalog_number_is_empty'] = 'Y'
    
        if len(remarks['title']) < 15 and len(remarks['title']) > 0:
            remarks['Title_is_too_short'] = 'Y'
    
        if len(remarks['description']) < 30 and len(remarks['description']) > 0:
            remarks['Description_is_too_short'] = 'Y'
    
        if remarks['Domain'] == '':
            remarks['Domain_is_empty'] = 'Y'
    
        if remarks['title'] == '':
            remarks['Title_is_empty'] = 'Y'
    
        if remarks['description'] == '':
            remarks['Description_is_empty'] = 'Y'
    
        if 'disclosureLevel' in row and row['disclosureLevel'] == '':
            remarks['Disclosure_Level_is_empty'] = 'Y'
    
        if 'persona' in row and row['persona'] == '':
            remarks['Persona_is_empty'] = 'Y'
    
        if 'products' in row and (row['products'] is None or len(row['products']) == 0):
            remarks['Products_is_empty'] = 'Y'
    
        return remarks


    @staticmethod
    def process_dataframe(df,index_name,include_id, include_catalog_number):
        
        remarks_info = df.apply(DataQualityProcessor.generate_remarks, axis=1, include_id=include_id, include_catalog_number=include_catalog_number)
        df_remarks = pd.DataFrame(remarks_info.tolist())
        
        # Additional processing
        df_remarks["Index_Name"] = index_name
        df_remarks["Source"] = index_name.split('.')[0]
        df_remarks['contentUpdateDate'] = pd.to_datetime(df_remarks['contentUpdateDate']).dt.strftime('%Y-%m-%d %H:%M:%S')
        df_remarks.fillna('', inplace=True)
        df_remarks['Contents'] = df_remarks.apply(lambda row: {
            'title': row['title'],
            'products': row['products'],
            'persona': row['persona'],
            'description': row['description'],
            'contentUpdateDate': row['contentUpdateDate'],
            'Domain': row['Domain']
        }, axis=1)
        df_remarks['Contents'] = df_remarks['Contents'].apply(lambda x: str(x))
        df_remarks=df_remarks.drop(columns=['products', 'persona'])
        # Select and order columns

        # Check for issues and create 'contains_issue' column
        columns_to_check = [
            "Title_is_too_short",
            "Description_is_too_short",
            "Domain_is_empty",
            "Title_is_empty",
            "Description_is_empty",
            "Disclosure_Level_is_empty",
            "Persona_is_empty",
            "Products_is_empty"
        ]
        if index_name.split('.')[0] =='kz':
            columns_to_check.append('catalog_number_is_empty')

        # Create the 'issues' column
        df_remarks['contains_issue'] = df_remarks[columns_to_check].apply(lambda row: 'Y' if 'Y' in row.values else 'N', axis=1)
        
        if 'kaas' in index_name:
            mask = df_remarks['documentID'].str.contains('ish') & (df_remarks['Title_is_too_short'] == 'Y')
            # Update the 'Title_is_too_short' column to 'N' for the filtered records
            df_remarks.loc[mask, 'Title_is_too_short'] = 'N'
        
        return df_remarks

    @staticmethod
    def generate_qc_data_log(test_df):
        index_name = 'kz.v6-embed'
        columns_to_check = [
            "Title_is_too_short",
            "Description_is_too_short",
            "Domain_is_empty",
            "Title_is_empty",
            "Description_is_empty",
            "Disclosure_Level_is_empty",
            "Persona_is_empty",
            "Products_is_empty",
            "contains_issue"
        ]
        
        # Append 'catalog_number_is_empty' if index_name starts with 'kz'
        if index_name.startswith('kz'):
            columns_to_check.append('catalog_number_is_empty')
        
        # Create dictionary to store counts using lambda
        count_dict = {
        col: test_df[col].value_counts().get('Y', 0) 
        for col in columns_to_check 
        if col in test_df.columns
        } 
        
        date_time = datetime.datetime.now()
        count_dict['datetime'] = date_time.strftime('%Y-%m-%d %H:%M:%S')
        count_dict['date'] = date_time.strftime('%Y-%m-%d')
        count_dict['source'] = index_name.split('.')[0]
        
        return count_dict

    @staticmethod
    def index_data_to_opensearch(df,index_name,qc_data_log):
        # Connect to OpenSearch
        config1 = OpenSearchConnectConfigStg()
        timeout_seconds = 160
        client = OpenSearch(
            hosts=[{'host': config1.host, 'port': config1.port}],
            http_compress=True,
            http_auth=config1.auth,
            use_ssl=config1.use_ssl,
            verify_certs=config1.verify_certs,
            timeout=timeout_seconds
        )
    
        # Retrieve data, preprocess, and convert to dictionary
    
        dict_list = df.to_dict(orient='records')
        print(dict_list)
        # Initialize counters and lists
        successful_records = 0
        failed_records = []
    
        # Prepare actions for bulk indexing
        actions = [
            {
                "_index": index_name,
                "_id": record["documentID"],
                "_op_type": "update",
                "doc": record,
                "doc_as_upsert": True  # Upsert if document doesn't exist
            }
            for record in dict_list
        ]

        try:
            success, _ = bulk(client, actions=actions)
            successful_records = success
        except Exception as e:
            # Log and store failed records
            for record in dict_list:
                failed_records.append((record, str(e), index_name.split('.')[0]))
                # logging.error(f"Failed to index document {record['_id']}: {e}")
        try:
            client.index(index='data_quality_check_datalog', body=qc_data_log)
        except Exception as e:
            raise
        return successful_records, len(failed_records)

In [18]:
test_df=DataQualityProcessor.process_dataframe(df1,'kz.v6-embed',False,True)

In [19]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9781 entries, 0 to 9780
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   documentID                 9781 non-null   int64 
 1   title                      9781 non-null   object
 2   description                9781 non-null   object
 3   contentUpdateDate          9781 non-null   object
 4   Domain                     9781 non-null   object
 5   Doc_Status                 9781 non-null   object
 6   language                   9781 non-null   object
 7   Title_is_too_short         9781 non-null   object
 8   Description_is_too_short   9781 non-null   object
 9   Domain_is_empty            9781 non-null   object
 10  Title_is_empty             9781 non-null   object
 11  Description_is_empty       9781 non-null   object
 12  Disclosure_Level_is_empty  9781 non-null   object
 13  Persona_is_empty           9781 non-null   object
 14  Products

In [20]:
qc_data_log=DataQualityProcessor.generate_qc_data_log(test_df)
qc_data_log

{'Title_is_too_short': 216,
 'Description_is_too_short': 3474,
 'Domain_is_empty': 0,
 'Title_is_empty': 0,
 'Description_is_empty': 1638,
 'Disclosure_Level_is_empty': 0,
 'Persona_is_empty': 0,
 'Products_is_empty': 2177,
 'contains_issue': 8525,
 'catalog_number_is_empty': 5136,
 'datetime': '2024-10-07 12:47:13',
 'date': '2024-10-07',
 'source': 'kz'}

In [21]:
DataQualityProcessor.index_data_to_opensearch(test_df,'data_quality_check_iteration',qc_data_log)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.wa

(9781, 0)

In [22]:
from opensearchpy.helpers import scan
from opensearchpy import OpenSearch, helpers
class OpenSearchManager:
    config1=OpenSearchConnectConfigStg()
    client = OpenSearch(
                hosts=[{'host': config1.host, 'port': config1.port}],
                http_compress=True,
                http_auth=config1.auth,
                use_ssl=config1.use_ssl,
                verify_certs=config1.verify_certs
            )
    @staticmethod
    def fetch_and_update_missing_documents(index_name, document_id_series):
        documents_to_update = []
        query = {
            "query": {
                "match_all": {}
            }
        }

        # Fetch all documents from the index
        results = scan(OpenSearchManager.client, query=query, index=index_name)

        # Extract the document IDs from the DataFrame for comparison
        df_document_ids = document_id_series.astype(str).tolist()

        # Iterate over the fetched results
        for res in results:
            # If the documentID is not found in the DataFrame, mark it as 'rejected'
            if res['_id'] not in df_document_ids:
                # Remove the 'query_embedding' field if it exists
                res['_source'].pop('query_embedding', None)
                res['_source']['Doc_Status'] = 'deleted'
                documents_to_update.append(res)

        # Prepare the documents for bulk update
        actions = [
            {
                "_op_type": "update",
                "_index": index_name,
                "_id": doc['_id'],
                "doc": doc['_source']
            }
            for doc in documents_to_update
        ]

        # Perform the bulk update
        helpers.bulk(OpenSearchManager.client, actions)

        return f"{len(documents_to_update)} documents updated with 'deleted' in Doc_Status and 'query_embedding' removed in index '{index_name}'"

    @staticmethod
    def clean_dashboard_data(index_name,source,test_df):
        # index_name='data_quality_check_iteration'
        # source='kaas'
        query = {
            "query": {
                "match_all": {}
            }
        }
        results = scan(OpenSearchManager.client, query=query, index=index_name)
        test = [res['_source'] for res in results]
        cd_df = pd.DataFrame(test)
        cd_df = cd_df[cd_df['Source'] == source]
        
        # Identify unmatched records
        df_document_ids = test_df['documentID'].astype(str).tolist()
        cd_df['documentID'] = cd_df['documentID'].astype(str)
        
        # Records in cd_df but not in test_df
        unmatched_records = cd_df[~cd_df['documentID'].isin(df_document_ids)]
        actions = [
            {
                "_op_type": "update",
                "_index": index_name,
                "_id": row['documentID'],
                "doc": {
                    "Doc_Status": "deleted"
                }
            }
            for _, row in unmatched_records.iterrows()
        ]
        
        # Execute the bulk update
        if actions:
            helpers.bulk(OpenSearchManager.client, actions)
        return f"{len(actions)} documents updated with 'deleted' in Doc_Status in '{index_name}'"

    @staticmethod
    def fetch_all_documents(index_name):
        """Fetches all documents from the specified index."""
        query = {"query": {"match_all": {}}}
        results = scan(OpenSearchManager.client, query=query, index=index_name)
        return results
        
    @staticmethod
    def process_and_delete_records(index_name):
        """Fetches, filters, and deletes records with Doc_Status 'deleted'."""
        results = OpenSearchManager.fetch_all_documents(index_name)
        actions = []

        for record in results:
            if record["_source"].get("Doc_Status") == "deleted":
                actions.append(
                    {"_op_type": "delete", "_index": index_name, "_id": record["_id"]}
                )

        if actions:
            bulk(OpenSearchManager.client, actions)
            return f"{len(actions)} documents deleted in '{index_name}'"

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [23]:
import time
start_time = time.time()
print(OpenSearchManager.fetch_and_update_missing_documents('kz.v6-embed', df1['documentID']))
end_time=time.time()
print('time_taken',end_time-start_time)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghr

0 documents updated with 'deleted' in Doc_Status and 'query_embedding' removed in index 'kz.v6-embed'
time_taken 13.253093004226685


In [24]:
import time
start_time = time.time()
print(OpenSearchManager.clean_dashboard_data('data_quality_check_iteration','kz',test_df))
end_time=time.time()
print('time_taken',end_time-start_time)

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghrejqchy.us-west-2.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'vpc-ks-stage-opensearch-es-a6nqrfnukskuiliztghr

0 documents updated with 'deleted' in Doc_Status in 'data_quality_check_iteration'
time_taken 8.194143772125244


In [92]:
import time
start_time = time.time()
print(OpenSearchManager.process_and_delete_records('kaas.v5.dev'))
print(OpenSearchManager.process_and_delete_records('docebo.v5.dev'))
print(OpenSearchManager.process_and_delete_records('kz.v6-embed'))
print(OpenSearchManager.process_and_delete_records('data_quality_check_iteration'))
end_time=time.time()
print('time_taken',end_time-start_time)

None
None
None
355 documents deleted in 'data_quality_check_iteration'
time_taken 30.939716577529907


In [ ]:
from langchain.schema import Document

def send_to_Opensearch(df):
    # Create Document objects from the DataFrame
    documents = [
        Document(page_content=row['text'], metadata=row['metadata'])
        for _, row in df.iterrows()
    ]
    
    docsearch = OpenSearchVectorSearch.from_documents(
        documents,
        CustomEmbeddingFunction(),
        opensearch_url='vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com:443',
        http_auth=('ks-stg-admin', 'K3!0dg_7Hc'),
        timeout=900,
        use_ssl=True,
        verify_certs=False,
        ssl_assert_hostname=False,
        ssl_show_warn=False,
        index_name="knowledge-ph2-2",
        engine="Lucene",
        method="hnsw",
        space_type="l2",
        bulk_size=10000
    )

# Example of using the function with your DataFrame
# Assuming df is your DataFrame with 'page_content' and 'metadata' columns


In [96]:
chunk_size = 1000  # Define the size of each chunk
start = 0
end = 50290
for i in range(start, end, chunk_size):
    # Define the start and end of the chunk
    chunk_start = i
    chunk_end = min(i + chunk_size, end)  # Ensure we don't exceed 54000

    # Get the chunk of data from the DataFrame
    data_chunk = dataframe.iloc[chunk_start:chunk_end]

    # Send the chunk to OpenSearch
    send_to_Opensearch(data_chunk)

    print(f"Sent records from {chunk_start} to {chunk_end}")

Sent records from 0 to 1000
Sent records from 1000 to 2000
Sent records from 2000 to 3000
Sent records from 3000 to 4000
Sent records from 4000 to 5000
Sent records from 5000 to 6000
Sent records from 6000 to 7000
Sent records from 7000 to 8000
Sent records from 8000 to 9000
Sent records from 9000 to 10000
Sent records from 10000 to 11000
Sent records from 11000 to 12000
Sent records from 12000 to 13000
Sent records from 13000 to 14000
Sent records from 14000 to 15000
Sent records from 15000 to 16000
Sent records from 16000 to 17000
Sent records from 17000 to 18000
Sent records from 18000 to 19000
Sent records from 19000 to 20000
Sent records from 20000 to 21000
Sent records from 21000 to 22000
Sent records from 22000 to 23000
Sent records from 23000 to 24000
Sent records from 24000 to 25000
Sent records from 25000 to 26000
Sent records from 26000 to 27000
Sent records from 27000 to 28000
Sent records from 28000 to 29000
Sent records from 29000 to 30000
Sent records from 30000 to 31000

In [25]:
# parallel processing send to opensearch
from langchain.schema import Document
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.schema import Document
import os
from typing import List
import pymupdf4llm
from chromadb import EmbeddingFunction, Documents
from langchain.vectorstores import elasticsearch
#from chromadb import EmbeddingFunction, Documents, Embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.retrievers import ElasticSearchBM25Retriever
from langchain_community.vectorstores import OpenSearchVectorSearch, Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from openai import OpenAI
import requests
from typing import List
from langchain_core.embeddings import Embeddings
# Function to send documents to OpenSearch in parallel
def send_chunk_to_opensearch(df_chunk):
    try:
        documents = [
            Document(page_content=row['text'], metadata=row['metadata'])
            for _, row in df_chunk.iterrows()
            if row['text'] is not None and row['text'].strip() != ""
        ]
                
        # Initialize OpenSearch vector search client
        docsearch = OpenSearchVectorSearch.from_documents(
            documents,
            CustomEmbeddingFunction(),
            opensearch_url='vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com:443',
            http_auth=('ks-stg-admin', 'K3!0dg_7Hc'),
            timeout=900,
            use_ssl=True,
            verify_certs=False,
            ssl_assert_hostname=False,
            ssl_show_warn=False,
            index_name="knowledge-ph2-3",
            engine="Lucene",
            method="hnsw",
            space_type="l2",
            bulk_size=10000
        )
        
        return True  # Return success status after uploading the chunk
    except Exception as e:
        print(f"Error processing chunk: {e}")
        return False  # Return failure status if an error occurs

# Main function to handle parallel processing
def send_to_opensearch_parallel(df, max_workers=10, chunk_size=500):
    # Split the DataFrame into chunks
    chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    
    # Use ThreadPoolExecutor to parallelize the operation
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(send_chunk_to_opensearch, chunk): chunk for chunk in chunks}
        
        # Collect results as they complete
        for future in as_completed(futures):
            chunk_id = futures[future]
            success = future.result()
            if success:
                print(f"Chunk {chunk_id} processed successfully")
            else:
                print(f"Chunk {chunk_id} processing failed")

# Example usage
print(datetime.datetime.now())
send_to_opensearch_parallel(df_test, max_workers=10, chunk_size=200)
print(datetime.datetime.now())

2024-10-07 12:49:08.484296


NameError: name 'df_test' is not defined

In [26]:
import os
from typing import List
import pymupdf4llm
from chromadb import EmbeddingFunction, Documents
from langchain.vectorstores import elasticsearch
#from chromadb import EmbeddingFunction, Documents, Embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.retrievers import ElasticSearchBM25Retriever
from langchain_community.vectorstores import OpenSearchVectorSearch, Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from openai import OpenAI
import requests
from typing import List
from langchain_core.embeddings import Embeddings

def generate_embeddings(text):
    # Make sure to `pip install openai` first
    client = OpenAI(base_url="http://15.43.5.213:11434/v1", api_key="ollama")
    model="nomic-embed-text:latest"
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        return list(map(generate_embeddings, texts))

    def embed_query(self, query: str) -> List[float]:
        return generate_embeddings(query)

    def embed_documents(self, texts: Documents) -> Embeddings:
        return list(map(generate_embeddings, texts))


# file="C:\Users\KuBP064\Downloads\pdf_8674677_en-US-4.pdf"

def split_Md(file):
    markdown_splitter = pymupdf4llm.to_markdown(file,write_images=True)
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    markdown_document = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,strip_headers=False)
    md_header_splits = markdown_document.split_text(markdown_splitter)
    for split in md_header_splits:
        split.metadata["source"]=file
        #print(split)
    return md_header_splits
from langchain_community.document_loaders import PyPDFLoader
def split_Page(file):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    return pages

def send_to_Opensearch(docs):
    docsearch = OpenSearchVectorSearch.from_documents(
        docs,
        CustomEmbeddingFunction(),
        opensearch_url="https://vpc-kcs-dev-hzczani3hzhcymj5tomuqth26y.us-west-2.es.amazonaws.com:443",
        http_auth=("kcsdevadmin", "S4e7Epr4T_O*HAkuW1ha"),
        timeout=300,
        use_ssl = True,
        verify_certs = False,
        ssl_assert_hostname = False,
        ssl_show_warn = False,
        index_name="knowledge6",
        engine="Lucene",
        method="hnsw",
        space_type="l2",
        bulk_size=10000
    )

'''docsearch = OpenSearchVectorSearch(
    embedding_function=CustomEmbeddingFunction(),
    opensearch_url="https://vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com:443",
    http_auth=("kcs-admin", "!QAZ2wsx"),
    timeout=300,
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
    index_name="knowledge1",
)

query = "Give me the requirments for Site prep guide?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)
print(docs[1].page_content)


filter_kwargs = {'filter': "8120-6352"}  ## query is just an OpenSearch query

#doc_retriever = docsearch.search("8120-6352")
#print(doc_retriever)'''

'docsearch = OpenSearchVectorSearch(\n    embedding_function=CustomEmbeddingFunction(),\n    opensearch_url="https://vpc-kcs-isearch-wao6ffx5mlzga2q2heyie7jigu.us-west-2.es.amazonaws.com:443",\n    http_auth=("kcs-admin", "!QAZ2wsx"),\n    timeout=300,\n    use_ssl=True,\n    verify_certs=False,\n    ssl_assert_hostname=False,\n    ssl_show_warn=False,\n    index_name="knowledge1",\n)\n\nquery = "Give me the requirments for Site prep guide?"\ndocs = docsearch.similarity_search(query)\nprint(docs[0].page_content)\nprint(docs[1].page_content)\n\n\nfilter_kwargs = {\'filter\': "8120-6352"}  ## query is just an OpenSearch query\n\n#doc_retriever = docsearch.search("8120-6352")\n#print(doc_retriever)'

In [65]:
text='im the one'
print(generate_embeddings(text))

APITimeoutError: Request timed out.

In [27]:
# from langchain.document_loaders import PyPDFLoader
import requests
from io import BytesIO
from PyPDF2 import PdfReader

def split_Page1(url):
    try:
        # Download the file from the provided URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Load the PDF content using BytesIO
            file_stream = BytesIO(response.content)
            
            try:
                # Attempt to load and process the PDF using PyPDF2
                reader = PdfReader(file_stream)
                pages = [page.extract_text() for page in reader.pages]
                return pages
            except Exception as pdf_error:
                print(f"Failed to process the file as a PDF: {pdf_error}")
                return None
        else:
            print(f"Failed to fetch URL: {url}")
            return None
    except Exception as e:
        print(f"Error processing URL: {url}. Error: {e}")
        return None
        
def preprocessing(df, url_column='preSignedUrl'):
    # List to hold records
    records = []
    
    # Iterate over the DataFrame
    for idx, row in df.iterrows():
        # Extract pages from the URL
        pages = split_Page1(row[url_column])
        
        # Continue if pages were successfully extracted
        if pages is not None:
            # Create metadata dictionary from other columns
            metadata = row.drop(url_column).to_dict()
            
            # Create a record for each page
            for page_num, page in enumerate(pages, start=1):
                # Add page number to the metadata
                meta_with_page_num = metadata.copy()
                meta_with_page_num['page_number'] = page_num
                
                # Append the record to the list
                records.append({
                    'metadata': meta_with_page_num,
                    'text': page
                })
    
    # Return a new DataFrame with separate records for each page
    return pd.DataFrame(records)

In [329]:
# langchain pdf reader
import tempfile
import os
from langchain.document_loaders import PyPDFLoader

def split_Page2(url):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        temp_file.write(requests.get(url).content)  # Assuming you're fetching PDF content from a URL
        temp_file.flush()
        temp_file_path = temp_file.name

    try:
        # Ensure file is accessible
        loader = PyPDFLoader(temp_file_path)
        pages = loader.load_and_split()
    finally:
        # Clean up temp file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

    return pages



In [28]:
# parallel processing
import pandas as pd
import requests
from io import BytesIO
from PyPDF2 import PdfReader
from concurrent.futures import ThreadPoolExecutor, as_completed

def split_Page1(url):
    try:
        # Download the file from the provided URL
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Load the PDF content using BytesIO
            file_stream = BytesIO(response.content)
            
            try:
                # Attempt to load and process the PDF using PyPDF2
                reader = PdfReader(file_stream)
                pages = [page.extract_text() for page in reader.pages]
                return pages
            except Exception as pdf_error:
                print(f"Failed to process the file as a PDF: {pdf_error}")
                return None
        else:
            print(f"Failed to fetch URL: {url}")
            return None
    except Exception as e:
        print(f"Error processing URL: {url}. Error: {e}")
        return None
        
def process_row(row, url_column):
    # Extract pages from the URL
    pages = split_Page1(row[url_column])
    
    # If pages were successfully extracted, create records
    if pages is not None:
        # Create metadata dictionary from other columns
        metadata = row.drop(url_column).to_dict()
        
        # Create a record for each page
        records = []
        for page_num, page in enumerate(pages, start=1):
            # Add page number to the metadata
            meta_with_page_num = metadata.copy()
            meta_with_page_num['page_number'] = page_num
            
            # Append the record to the list
            records.append({
                'metadata': meta_with_page_num,
                'text': page
            })
        return records
    return []

def preprocessing(df, url_column='preSignedUrl', max_workers=5):
    # List to hold all records
    all_records = []
    
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit each row for processing in parallel
        future_to_row = {executor.submit(process_row, row, url_column): idx for idx, row in df.iterrows()}
        
        # Gather results as they complete
        for future in as_completed(future_to_row):
            try:
                records = future.result()
                if records:
                    all_records.extend(records)
            except Exception as e:
                print(f"Error processing row: {e}")
    
    # Return a new DataFrame with separate records for each page
    return pd.DataFrame(all_records)


In [30]:
result=kzloader()
df=result.make_api_request()
df1=result.preprocess_data(df)
print(df1.shape)
df_doc=df1[df1['contentType']=='Document']

In [32]:
start=datetime.datetime.now()
print(start)
# df_check=preprocessing(df_doc[:4000], url_column='preSignedUrl', max_workers=10)
df_check1=preprocessing(df_doc[4000:], url_column='preSignedUrl', max_workers=10)
end=datetime.datetime.now()
print(end)
time_taken = (end - start).total_seconds() / 60
print(f"Time taken: {time_taken:.2f} minutes")

2024-10-07 13:59:22.013640
Failed to process the file as a PDF: Invalid Elementary Object starting with b'\xd8' @320405: b'\x9b7o\x90\x15\xf3\xf2\n\xb9\xba:GGG\x01I\x00EI \xd8\nEI Q\nq 0 12 -1 0 5363 680.996 cm\nBI\n/CS/RGB\n/W 12\n/H 1\n/BP'
Error processing URL: . Error: Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error processing URL: . Error: Invalid URL '': No scheme supplied. Perhaps you meant https://?
Failed to process the file as a PDF: EOF marker not found
Failed to process the file as a PDF: EOF marker not found
Error processing URL: . Error: Invalid URL '': No scheme supplied. Perhaps you meant https://?
Failed to process the file as a PDF: EOF marker not found


[0, IndirectObject(8033, 0, 2400399992016)]
[0, IndirectObject(8028, 0, 2400399992016)]
[0, IndirectObject(8023, 0, 2400399992016)]
[0, IndirectObject(205, 0, 2400782191760)]


Failed to process the file as a PDF: EOF marker not found


[0, IndirectObject(8257, 0, 2399929844432)]
[0, IndirectObject(8252, 0, 2399929844432)]
[0, IndirectObject(8247, 0, 2399929844432)]
[0, IndirectObject(8242, 0, 2399929844432)]
[0, IndirectObject(8237, 0, 2399929844432)]
[0, IndirectObject(8232, 0, 2399929844432)]
[0, IndirectObject(8227, 0, 2399929844432)]
[0, IndirectObject(8222, 0, 2399929844432)]
[0, IndirectObject(8217, 0, 2399929844432)]
[0, IndirectObject(8212, 0, 2399929844432)]
[0, IndirectObject(8207, 0, 2399929844432)]
[0, IndirectObject(8202, 0, 2399929844432)]
[0, IndirectObject(8197, 0, 2399929844432)]
[0, IndirectObject(8192, 0, 2399929844432)]
[0, IndirectObject(8187, 0, 2399929844432)]
[0, IndirectObject(8182, 0, 2399929844432)]
[0, IndirectObject(8177, 0, 2399929844432)]
[0, IndirectObject(8172, 0, 2399929844432)]
[0, IndirectObject(8167, 0, 2399929844432)]
[0, IndirectObject(8162, 0, 2399929844432)]
[0, IndirectObject(8157, 0, 2399929844432)]
[0, IndirectObject(8152, 0, 2399929844432)]


Failed to process the file as a PDF: EOF marker not found
Failed to process the file as a PDF: EOF marker not found


Failed to process the file as a PDF: negative seek value -1
Failed to process the file as a PDF: negative seek value -1
Failed to process the file as a PDF: EOF marker not found
2024-10-07 16:36:55.723670
Time taken: 157.56 minutes


In [35]:
# print(start,end)
df_check.info()
df_check1.info()

# df_check1[df_check1['text'].apply(lambda x: len(x)>1)]
kz_df = pd.concat([df_check, df_check1], ignore_index=True)
kz_df=kz_df[kz_df['text'].apply(lambda x: len(x)>0)]
# kz_df = kz_df[kz_df['text'].apply(lambda x: isinstance(x, str) and len(x) > 0)]
# kz_df[kz_df['text'].apply(lambda x: len(x)>0)]
kz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81593 entries, 0 to 81592
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   metadata  81593 non-null  object
 1   text      81593 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117305 entries, 0 to 117304
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   metadata  117305 non-null  object
 1   text      117305 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 196116 entries, 0 to 198897
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   metadata  196116 non-null  object
 1   text      196116 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [ ]:
def generate_embeddings(text):
    # Make sure to `pip install openai` first
    client = OpenAI(base_url="http://15.43.5.213:11434/v1", api_key="ollama")
    model="nomic-embed-text:latest"
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding
kz_df['vector'] = kz_df['text'].apply(lambda x: generate_embeddings(x) if pd.notnull(x) else None)


In [ ]:
print(datetime.datetime.now())

In [ ]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm  # To show a progress bar, optional


# Define a helper function to process a single row of the DataFrame
def process_row(text):
    if pd.notnull(text):  # Only process if text is not null
        return generate_embeddings(text)
    else:
        return None

# Function to apply concurrent processing
def generate_embeddings_concurrently(df, text_column='text', vector_column='vector', max_workers=5):
    # Use ThreadPoolExecutor to parallelize the operation
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Map each row's 'text' field to the `process_row` function
        # tqdm is used to display a progress bar, you can remove it if not needed
        df[vector_column] = list(tqdm(executor.map(process_row, df[text_column]), total=len(df)))
    return df

# Example usage
df = generate_embeddings_concurrently(df, text_column='text', vector_column='vector', max_workers=10)

# df now contains a new 'vector' column with the embeddings


In [28]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Find semantic boundaries based on cosine similarity of embeddings
def find_semantic_boundaries(embeddings, threshold=0.75):
    """Detects boundaries where semantic similarity falls below the threshold."""
    boundaries = [0]  # Start with the first section as a boundary
    for i in range(1, len(embeddings)):
        similarity = cosine_similarity([embeddings[i-1]], [embeddings[i]])[0][0]
        if similarity < threshold:  # Threshold to detect semantic change
            boundaries.append(i)
    boundaries.append(len(embeddings))  # End of the document as a boundary
    return boundaries


# Step 2: Group text based on semantic boundaries
def group_text_into_semantic_chunks(df, boundaries):
    semantic_chunks = []
    for i in range(len(boundaries) - 1):
        # Group text from pages between boundary i and i+1
        chunk = " ".join(df['text'][boundaries[i]:boundaries[i+1]].tolist())
        semantic_chunks.append(chunk)
    return semantic_chunks


# Main function to process the DataFrame and chunk text semantically
def process_semantic_chunking(df, threshold=0.75):
    results = []
    
    # Group by documentID to process each document independently
    for doc_id, group in df.groupby('documentID'):
        # Get embeddings and text for the current document
        page_embeddings = group['vectorfield'].tolist()
        
        # Step 1: Find semantic boundaries
        boundaries = find_semantic_boundaries(page_embeddings, threshold)
        
        # Step 2: Group text into semantic chunks
        semantic_chunks = group_text_into_semantic_chunks(group, boundaries)
        
        # Step 3: Store the result
        for chunk in semantic_chunks:
            results.append({
                'documentID': doc_id,
                'semantic_chunk': chunk
            })
    
    # Create a new DataFrame for semantic chunks
    return pd.DataFrame(results)


# Example DataFrame structure (replace with your actual DataFrame)
data = {
    'documentID': ['doc1'] * 5,  # Example: 5 pages for doc1
    'text': [
        "Text from page 1.",
        "Text from page 2.",
        "Text from page 3.",
        "Text from page 4.",
        "Text from page 5."
    ],
    'vectorfield': [
        [0.1, 0.2, 0.3],  # Example embeddings (replace with real embeddings)
        [0.11, 0.21, 0.29],
        [0.5, 0.6, 0.7],
        [0.52, 0.58, 0.72],
        [0.9, 1.0, 1.1]
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Process semantic chunking for the DataFrame
semantic_chunks_df = process_semantic_chunking(df, threshold=0.75)


In [29]:
df_test['text'] = df_test['metadata'].apply(lambda x: x.get('ti_desc_prod', '')) +'. '+ df_test['text']

In [30]:
df_test[df_test['metadata'].apply(lambda x: x.get('documentID')==240997)].to_dict(orient='records')

[{'metadata': {'documentID': 240997,
   'title': 'HP Indigo Mixed Reality Procedures Booklet',
   'provider': 'Storage-Service',
   'format': 'pdf',
   'language': 'en',
   'uuid': 'b3fb12b2-a6cd-4c84-b2e5-fe463f58dccf',
   'description': ' xR-0014',
   'contentType': 'Document',
   'Domain': 'Indigo',
   'contentUpdateDate': '2024-07-16 10:06:24',
   'namedAsset': 'bf3b11f2-6e7a-451e-9c59-d9a8af162cbf',
   'lowestRolesList': 'Press Operator',
   'catalog_number': 'xR-0014',
   'asset_state': {'name': 'Regular', 'numericOrder': 99, 'dueDate': None},
   'publisher': {'id': 1, 'name': 'HP Indigo'},
   'ratingResponse': {'number_of_votes': 0, 'rating': 0.0},
   'products': ['HP Indigo 6900 Digital Press',
    'HP Indigo 100K Digital Press',
    'HP Indigo 6K Digital Press',
    'HP Indigo 12000 Digital Press',
    'HP Indigo 12000 HD Digital Press'],
   'persona': 'Operator',
   'renderLink': 'https://www.printos.com/knowledge-zone/#/view/asset/240997',
   'ti_desc_prod': 'HP Indigo Mixed

In [43]:
from opensearchpy.helpers import scan
from opensearchpy import OpenSearch, helpers
class OpenSearchConnectConfigDev:
    def __init__(self):
        self.host = 'vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com'       
        self.port = 443
        self.auth = ('ks-stg-admin', 'K3!0dg_7Hc')    
        self.use_ssl = True
        self.verify_certs = False
config1 = OpenSearchConnectConfigDev()
timeout_seconds = 160
client = OpenSearch(
    hosts=[{'host': config1.host, 'port': config1.port}],
    http_compress=True,
    http_auth=config1.auth,
    use_ssl=config1.use_ssl,
    verify_certs=config1.verify_certs,
    timeout=timeout_seconds
)
query = {
    "query": {
        "match_all": {}}}

results = scan(client, query=query, index='knowledge-ph2-2')

l_index=list({i['_id'] for i in results})

l_ingress = list({str(i) for i in kz_df['metadata'].apply(lambda x: x.get('documentID') if isinstance(x, dict) else None) if i is not None})

diff = [item for item in l_index if str(item).split('_')[0] in l_ingress]

actions=[]
for result in diff:
    actions.append({
        "_op_type": "delete",
        "_index": 'knowledge-ph2-2' ,
        "_id": result
    })
len(actions)
actions[0]
# if actions:
#     success, failed = bulk(client, actions)
#     print(f"Successfully deleted {success} documents.")
#     if failed:
#         print(f"Failed to delete {failed} documents.")
# else:
#     print("No documents found to delete.")

C:\Users\KuBP064\AppData\Local\anaconda3\Lib\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://vpc-ks-dev-opensearch-es-oacgyoijjstnydxq2itjbftyvm.us-west-2.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(


NameError: name 'index_name' is not defined

In [ ]:
data = []

# Scan the OpenSearch index and extract relevant fields
results = scan(client, query=query, index='knowledge-ph2-2')
for i in results:
    vector_field = i['_source']['vector_field']
    document_id = i['_source']['metadata'].get('documentID')
    text = i['_source']['text']
    
    # Append the extracted fields as a tuple
    data.append((document_id, vector_field, text))

# Create a DataFrame from the extracted data
df_sem = pd.DataFrame(data, columns=['documentID', 'vector_field', 'text'])


In [67]:
def process_batch(batch):
    # Convert batch to DataFrame
    batch_df = pd.DataFrame(batch, columns=['documentID', 'vector_field', 'text'])
    
    # Return the batch DataFrame for accumulation
    return batch_df

# Scan the OpenSearch index in chunks
def scan_in_chunks(client, query, index, chunk_size=100):
    results = scan(client, query=query, index=index)
    batch = []
    all_batches = []  # List to accumulate all batch DataFrames

    for result in results:
        vector_field = result['_source']['vector_field']
        document_id = result['_source']['metadata'].get('documentID')
        text = result['_source']['text']
        
        # Add to current batch
        batch.append((document_id, vector_field, text))

        # If the batch size limit is reached, process the batch
        if len(batch) == chunk_size:
            # Process the current batch and collect the resulting DataFrame
            batch_df = process_batch(batch)
            all_batches.append(batch_df)  # Append the DataFrame to the list
            batch.clear()  # Clear the batch for the next set

    # Process any remaining results that didn't fill up the last batch
    if batch:
        batch_df = process_batch(batch)
        all_batches.append(batch_df)  # Append the last batch DataFrame

    # Concatenate all the DataFrames into a single final DataFrame
    final_df = pd.concat(all_batches, ignore_index=True)
    return final_df

# Usage





In [68]:
final_dataframe = scan_in_chunks(client, query=query, index='knowledge-ph2-2', chunk_size=100)

ConnectionError: ConnectionError(Unable to allocate output buffer.) caused by: MemoryError(Unable to allocate output buffer.)